# Number of OTM per Execution Date 

## Query

This analysis starts with the output of this query, which I constructed in PGAdmin and saved in the following file:
*number_otm_execution_date.sql*

## Reading-In Output of Query

The output of the above query is exported into *universe_per_expiration.csv* and we read it in here.

In [ ]:
import pandas as pd

Notice that this dataset consists of the number of OTM option that are in `otm_history` for each expiration on each execution date.

In [ ]:
df_otm = pd.read_csv('../data/universe_per_expiration.csv')
df_otm

,underlying,monthly,expiration,execution_date,last_trade_date,num_otm
0,DIA,False,2010-06-11,2010-06-04,2010-06-11,15
1,DIA,True,2010-06-19,2010-06-11,2010-06-18,15
2,DIA,False,2010-06-25,2010-06-18,2010-06-25,11
3,DIA,True,2010-07-17,2010-07-09,2010-07-16,13
4,DIA,True,2010-08-21,2010-08-13,2010-08-20,14
...,...,...,...,...,...,...
13169,XRT,False,2018-11-30,2018-11-23,2018-11-30,14
13170,XRT,False,2018-12-07,2018-11-30,2018-12-07,15
13171,XRT,False,2018-12-14,2018-12-07,2018-12-14,13
13172,XRT,True,2018-12-21,2018-12-14,2018-12-21,11


Let's see how many we have if we filter by `num_opt>=4`.

In [ ]:
df_otm.query('num_otm >= 4')

,underlying,monthly,expiration,execution_date,last_trade_date,num_otm
0,DIA,False,2010-06-11,2010-06-04,2010-06-11,15
1,DIA,True,2010-06-19,2010-06-11,2010-06-18,15
2,DIA,False,2010-06-25,2010-06-18,2010-06-25,11
3,DIA,True,2010-07-17,2010-07-09,2010-07-16,13
4,DIA,True,2010-08-21,2010-08-13,2010-08-20,14
...,...,...,...,...,...,...
13169,XRT,False,2018-11-30,2018-11-23,2018-11-30,14
13170,XRT,False,2018-12-07,2018-11-30,2018-12-07,15
13171,XRT,False,2018-12-14,2018-12-07,2018-12-14,13
13172,XRT,True,2018-12-21,2018-12-14,2018-12-21,11


## Counting underlyings while filtering for `num_otm`

Now we want to filter for underlyings which have at least 4 OTM options in the chain, and then count how many underlyings fit this criteria.  Notice that 6/11/2010 only has four (which you can inspect to be SPY, IWM, QQQ, DIA).  6/11/2010 has a lot more because 6/19/2010 is a monthly expiration.  When you start getting into 2018 then there are nearly 35 per expiration.

In [ ]:
df_num_underlying = \
    (
    df_otm.query('num_otm >= 4')
        .groupby(['monthly', 'expiration', 'execution_date', 'last_trade_date'])['num_otm'].count()
        .to_frame()
        .sort_values('execution_date')
        .reset_index()
        .rename(columns={'num_otm':'num_underlying'})
    )
df_num_underlying

,monthly,expiration,execution_date,last_trade_date,num_underlying
0,False,2010-06-11,2010-06-04,2010-06-11,4
1,True,2010-06-19,2010-06-11,2010-06-18,31
2,False,2010-06-25,2010-06-18,2010-06-25,4
3,False,2010-07-02,2010-06-25,2010-07-02,5
4,False,2010-07-09,2010-07-02,2010-07-09,6
...,...,...,...,...,...
442,False,2018-11-30,2018-11-23,2018-11-30,35
443,False,2018-12-07,2018-11-30,2018-12-07,35
444,False,2018-12-14,2018-12-07,2018-12-14,35
445,True,2018-12-21,2018-12-14,2018-12-21,35


Let's see how many execution dates have less than 4 underlyings.  Note that for `execution_date=2015-12-11` and `expiration=2015-12-2019` there is only one eligible underlying.  I did some investigation of the raw `option_price` table and I found that all underlyings except SPY have a expiration date of 2015-12-18.  I'll have to go and fix that, and keep some kind of running list of all manual modifications I make to the database.

In [ ]:
df_num_underlying.query('num_underlying < 4')

,monthly,expiration,execution_date,last_trade_date,num_underlying
288,True,2015-12-19,2015-12-11,2015-12-18,1


## Exporting to CSV

Exporting to CSV mainly because it's easier to read from a spreadsheet view.

In [ ]:
df_num_underlying.to_csv('../data/number_underlying_per_expiration.csv', index=False)